# Phase Estimation of Quantum Walks

In [1]:
#  Copyright 2023 Google LLC
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      https://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.

## Heisenberg limited phase estimation
Implements Heisenberg-Limited Phase Estimation of the Qubitized Quantum Walks as described in Section-II B. of [Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity](https://arxiv.org/abs/1805.03662)

In [2]:
import cirq
import numpy as np

from qualtran._infra.gate_with_registers import get_named_qubits
from qualtran.bloqs.qubitization_walk_operator import QubitizationWalkOperator
from qualtran.bloqs.qubitization_walk_operator_test import get_walk_operator_for_1d_Ising_model, walk_operator_for_pauli_hamiltonian, get_1d_Ising_hamiltonian
from qualtran.bloqs.hubbard_model import get_walk_operator_for_hubbard_model
from qualtran.testing import assert_valid_bloq_decomposition, execute_notebook
from qualtran.bloqs.reflection_using_prepare_test import construct_gate_helper_and_qubit_order
from collections import Counter

/Users/tanujkhattar/opt/anaconda3/envs/qualtran2/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:34: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [3]:
from qualtran.bloqs.phase_estimation.lp_resource_state import LPResourceState
from qualtran.bloqs.phase_estimation.resource_state_qubitization_pe import QubitizationPE
from qualtran.bloqs.qft.qft_text_book import QFTTextBook
from qualtran.cirq_interop.testing import GateHelper

def get_resource_state(m: int):
    r"""Returns a state vector representing the resource state on m qubits from Eq.17 of Ref-1.

    Returns a numpy array of size 2^{m} representing the state vector corresponding to the state
    $$\n",
        \\sqrt{\\frac{2}{2^m + 1}} \\sum_{n=0}^{2^{m}-1} \\sin{\\frac{\\pi(n + 1)}{2^{m}+1}}\\ket{n}
    $$

    Args:
        m: Number of qubits to prepare the resource state on.
    
    Ref:
        1) [Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity]
            (https://arxiv.org/abs/1805.03662)
            Eq. 17
    """
    den = 1 + 2 ** m
    norm = np.sqrt(2 / den)
    return norm * np.sin(np.pi * (1 + np.arange(2**m)) / den)

def phase_estimation(walk: QubitizationWalkOperator, m: int) -> cirq.OP_TREE:
    """Heisenberg limited phase estimation circuit for learning eigenphase of `walk`.
    
    The method yields an OPTREE to construct Heisenberg limited phase estimation circuit 
    for learning eigenphases of the `walk` operator with `m` bits of accuracy. The 
    circuit is implemented as given in Fig.2 of Ref-1.
    
    Args:
        walk: Qubitization walk operator.
        m: Number of bits of accuracy for phase estimation. 
        
    Ref:
        1) [Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity]
            (https://arxiv.org/abs/1805.03662)
            Fig. 2
    """
    reflect = walk.reflect
    walk_regs = get_named_qubits(walk.signature)
    reflect_regs = {reg.name: walk_regs[reg.name] for reg in reflect.signature}
    
    reflect_controlled = reflect.controlled(control_values=[0])
    walk_controlled = walk.controlled(control_values=[1])

    m_qubits = [cirq.q(f'm_{i}') for i in range(m)]
    state_prep = cirq.StatePreparationChannel(get_resource_state(m), name='chi_m')

    yield state_prep.on(*m_qubits)
    yield walk.controlled(control_values=[0]).on_registers(**walk_regs, control=m_qubits[0])
    for i in range(1, m):
        yield reflect_controlled.on_registers(control=m_qubits[i], **reflect_regs)
        yield walk.on_registers(**walk_regs)
        walk = walk ** 2
        yield reflect_controlled.on_registers(control=m_qubits[i], **reflect_regs)
        
    yield cirq.qft(*m_qubits, inverse=True)

num_sites: int = 6
eps: float = 1e-2
m_bits: int = 4

circuit = cirq.Circuit(phase_estimation(get_walk_operator_for_1d_Ising_model(num_sites, eps), m=m_bits))
print(circuit)

m_0: ──────────chi_m[1]───@(0)─────────────────────────────────────────────────────────────qft^-1───
               │          │                                                                │
m_1: ──────────chi_m[2]───┼──────@(0)───────@(0)───────────────────────────────────────────#2───────
               │          │      │          │                                              │
m_2: ──────────chi_m[3]───┼──────┼──────────┼──────@(0)─────────@(0)───────────────────────#3───────
               │          │      │          │      │            │                          │
m_3: ──────────chi_m[4]───┼──────┼──────────┼──────┼────────────┼──────@(0)─────────@(0)───#4───────
                          │      │          │      │            │      │            │
selection0: ──────────────W──────R_L────W───R_L────R_L────W─────R_L────R_L────W─────R_L─────────────
                          │      │      │   │      │      │     │      │      │     │
selection1: ──────────────W──────R_L────W───

In [11]:
from typing import Union
from qualtran.bloqs.phase_estimation.lp_resource_state import LPResourceState
from qualtran.bloqs.reflection_using_prepare import ReflectionUsingPrepare
from qualtran.bloqs.phase_estimation.resource_state_qubitization_pe import QubitizationPE
from qualtran.bloqs.phase_estimation.kitaev_qpe_text_book import KitaevQPETextBook
from qualtran.bloqs.qft.qft_text_book import QFTTextBook
from qualtran.cirq_interop.testing import GateHelper
from qualtran.bloqs.basic_gates import Swap

gateset_to_decompose = cirq.Gateset(
    # QubitizationWalkOperator,
    # QubitizationPE,
    # ReflectionUsingPrepare,
    KitaevQPETextBook,
)

def keep(op: cirq.Operation):
    ret = op in gateset_to_decompose
    if op.gate is not None and isinstance(op.gate, cirq.ops.raw_types._InverseCompositeGate):
        ret |= op.gate._original in gateset_to_decompose
    return not ret

def phase_estimation(walk: Union[KitaevQPETextBook, QubitizationPE], m: int, keep=lambda op: not isinstance(op.gate, QubitizationPE)) -> cirq.OP_TREE:
    """Heisenberg limited phase estimation circuit for learning eigenphase of `walk`.
    
    The method yields an OPTREE to construct Heisenberg limited phase estimation circuit 
    for learning eigenphases of the `walk` operator with `m` bits of accuracy. The 
    circuit is implemented as given in Fig.2 of Ref-1.
    
    Args:
        walk: Qubitization walk operator.
        m: Number of bits of accuracy for phase estimation. 
        
    Ref:
        1) [Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity]
            (https://arxiv.org/abs/1805.03662)
            Fig. 2
    """
    # state_prep = LPResourceState(m)
    # qpe = QubitizationPE(walk, m)
    qpe = KitaevQPETextBook(walk, m)
    print(walk, qpe)

    quregs = GateHelper(qpe).quregs

    # yield state_prep.on(*quregs['phase_reg'])
    
    context = cirq.DecompositionContext(cirq.GreedyQubitManager(prefix="ancilla", maximize_reuse=True))
    op = qpe.on_registers(**quregs)
    yield cirq.decompose(op, keep=keep, on_stuck_raise=None, context=context)

    # yield QFTTextBook(m, with_reverse=True).adjoint().on_registers(q=quregs['phase_reg'])
    # Reverse qubits after inverse QFT
    m_mid = m_bits // 2
    phase_qubits = quregs['phase_reg']
    print(m_bits, m_mid, phase_qubits[:m_mid], phase_qubits[m_bits-1 : m_mid : -1][:m_mid])
    # yield Swap(m_mid).on_registers(
    #     x=phase_qubits[:m_mid], y=phase_qubits[-m_mid:][::-1]
    # )
    # yield qft.on(*quregs['phase_reg'])
    # yield cirq.measure(*quregs['phase_reg'])

In [12]:
from qualtran.bloqs.basic_gates import Hadamard, OnEach
from qualtran.bloqs.state_preparation import PrepareUniformSuperposition
from qualtran.bloqs.multiplexers.select_pauli_lcu import SelectPauliLCU
import scipy


num_sites, eps, m_bits = 4, 0.1, 8
ham = get_1d_Ising_hamiltonian(cirq.LineQubit.range(num_sites))

q = cirq.LineQubit.range(num_sites)
ham = cirq.PauliSum()
paulis = [cirq.X, cirq.Y, cirq.Z]
for i in range(num_sites):
    ham += paulis[i%3].on(q[i]) * paulis[i%3].on(q[(i + 1) % num_sites])
# ham /= np.sqrt(2**len(ham))
assert scipy.linalg.ishermitian(ham.matrix())


ham_coeff = [abs(ps.coefficient.real) for ps in ham]
print()

prepare = PrepareUniformSuperposition(len(ham_coeff))

ham_dps = [ps.dense(q) for ps in ham]
select = SelectPauliLCU(
    (len(ham_coeff) - 1).bit_length(), select_unitaries=ham_dps, target_bitsize=num_sites
)

# walk = walk_operator_for_pauli_hamiltonian(ham, eps)
walk = QubitizationWalkOperator(select, prepare)

qpe_bloq = KitaevQPETextBook(walk, m_bits)

quregs = GateHelper(qpe_bloq).quregs
qubit_order = cirq.QubitOrder.explicit([*quregs['phase_reg'], *quregs['selection'], *quregs['target']], fallback=cirq.QubitOrder.DEFAULT)

qpe = cirq.Circuit(phase_estimation(walk, m=m_bits, keep=keep))

qpe



QubitizationWalkOperator(select=SelectPauliLCU(selection_bitsize=2, target_bitsize=4, select_unitaries=(cirq.DensePauliString('XXII', coefficient=(1+0j)), cirq.DensePauliString('IYYI', coefficient=(1+0j)), cirq.DensePauliString('IIZZ', coefficient=(1+0j)), cirq.DensePauliString('XIIX', coefficient=(1+0j))), control_val=None), prepare=PrepareUniformSuperposition(n=4, cvs=()), control_val=None, power=1) KitaevQPETextBook(unitary=QubitizationWalkOperator(select=SelectPauliLCU(selection_bitsize=2, target_bitsize=4, select_unitaries=(cirq.DensePauliString('XXII', coefficient=(1+0j)), cirq.DensePauliString('IYYI', coefficient=(1+0j)), cirq.DensePauliString('IIZZ', coefficient=(1+0j)), cirq.DensePauliString('XIIX', coefficient=(1+0j))), control_val=None), prepare=PrepareUniformSuperposition(n=4, cvs=()), control_val=None, power=1), m_bits=8)
8 4 [cirq.NamedQubit('phase_reg0') cirq.NamedQubit('phase_reg1')
 cirq.NamedQubit('phase_reg2') cirq.NamedQubit('phase_reg3')] [cirq.NamedQubit('phase_r

phase_reg0: ───H──────────────────────────────────────────────@───────Adjoint───
                                                              │       │
phase_reg1: ───H───────────────────────────────────────@──────┼───────q─────────
                                                       │      │       │
phase_reg2: ───H────────────────────────────────@──────┼──────┼───────q─────────
                                                │      │      │       │
phase_reg3: ───H─────────────────────────@──────┼──────┼──────┼───────q─────────
                                         │      │      │      │       │
phase_reg4: ───H───────────────────@─────┼──────┼──────┼──────┼───────q─────────
                                   │     │      │      │      │       │
phase_reg5: ───H─────────────@─────┼─────┼──────┼──────┼──────┼───────q─────────
                             │     │     │      │      │      │       │
phase_reg6: ───H───────@─────┼─────┼─────┼──────┼──────┼──────┼───────q─────────
                       │     │     │     │      │      │      │       │
phase_reg7: ───H───@───┼─────┼─────┼─────┼──────┼──────┼──────┼───────q─────────
                   │   │     │     │     │      │      │      │
selection0: ───────W───W─────W─────W─────W──────W──────W──────W─────────────────
                   │   │     │     │     │      │      │      │
selection1: ───────W───W─────W─────W─────W──────W──────W──────W─────────────────
                   │   │     │     │     │      │      │      │
target0: ──────────W───W─────W─────W─────W──────W──────W──────W─────────────────
                   │   │     │     │     │      │      │      │
target1: ──────────W───W─────W─────W─────W──────W──────W──────W─────────────────
                   │   │     │     │     │      │      │      │
target2: ──────────W───W─────W─────W─────W──────W──────W──────W─────────────────
                   │   │     │     │     │      │      │      │
target3: ──────────W───W^2───W^4───W^8───W^16───W^32───W^64───W^128─────────────

In [13]:
ham_coeff = [abs(ps.coefficient.real) for ps in ham]
qubitization_lambda = np.sum(ham_coeff)

g = GateHelper(walk)
L_state = np.zeros(2 ** len(g.quregs['selection']))
L_state[: len(ham_coeff)] = np.sqrt(ham_coeff / qubitization_lambda)

assert len(qpe.all_qubits()) < 23

sim = cirq.Simulator(dtype=np.complex128)

eigen_values, eigen_vectors = np.linalg.eigh(ham.matrix())

print(eigen_values)

for eig_idx, eig_val in enumerate(eigen_values):
    # Applying QPE to determine eigenvalue for walk operator W on initial state |L>|k>
    K_state = eigen_vectors[:, eig_idx].flatten()
    L_K = np.kron(L_state, K_state)
    L_K /= np.linalg.norm(L_K)
    prep_L_K = cirq.Circuit(
        cirq.StatePreparationChannel(L_K, name="PREP_L_K").on(*g.quregs['selection'], *g.quregs['target']),
    )
    qpe_with_init = prep_L_K + qpe
    assert len(qpe_with_init.all_qubits()) < 23
    # Final state: W|L>|k>|temp> with |temp> register traced out.
    # result = sim.run(qpe_with_init, repetitions=1)
    # print(result)
    # break
    counts = Counter()
    final_state = sim.simulate(qpe_with_init, qubit_order=qubit_order).final_state_vector
    samples = cirq.sample_state_vector(final_state, indices=[*range(m_bits)], repetitions=5000)
    counts = np.bincount(samples.dot(1 << np.arange(samples.shape[-1] - 1, -1, -1)))
    assert len(counts) <= 2**m_bits
    print(cirq.dirac_notation(K_state), np.argmax(counts), (np.argmax(counts) / 2 ** m_bits))
    phase = (np.argmax(counts) / 2 ** m_bits) * 2 * np.pi

    expected_one = np.arccos(eig_val / qubitization_lambda)
    expected_two = np.arccos(eig_val / qubitization_lambda) + np.pi
    # expected_one = np.cos(phase * 2 * np.pi )
    # expected_two = np.cos(phase * 2 * np.pi - np.pi/2)
    
    print(phase, eig_val / qubitization_lambda, expected_one, expected_two, eps)
    is_close = [
                np.allclose(phase, expected_one, atol=eps), 
                np.allclose(phase, expected_two, atol=eps), 
                np.allclose(2 * np.pi - phase, expected_one, atol=eps), 
                np.allclose(2 * np.pi - phase, expected_two, atol=eps), 
               ]
    # is_close = [
    #             np.allclose(eig_val / qubitization_lambda, expected_one, atol=eps), 
    #             np.allclose(eig_val / qubitization_lambda, -expected_one, atol=eps), 
    #             np.allclose(eig_val / qubitization_lambda, expected_two, atol=eps),
    #             np.allclose(eig_val / qubitization_lambda, -expected_two, atol=eps),
    #            ]
    print(is_close)

    if np.abs(eig_val) > 1e-6:
        assert np.any(is_close)
    # Overlap: <L|k|W|k|L> = E_{k} / lambda
    # overlap = np.vdot(L_K, final_state)
    # cirq.testing.assert_allclose_up_to_global_phase(
    #     overlap, eig_val / qubitization_lambda, atol=1e-6
    # )

[-2.73205081 -2.73205081 -2.73205081 -2.73205081 -0.73205081 -0.73205081
 -0.73205081 -0.73205081  0.73205081  0.73205081  0.73205081  0.73205081
  2.73205081  2.73205081  2.73205081  2.73205081]
0.32|0000⟩ + 0.01|0001⟩ - 0.01|0010⟩ + 0.01|0100⟩ + 0.45|0101⟩ + 0.44|0110⟩ - 0.61|1001⟩ - 0.16|1010⟩ + 0.01|1011⟩ - 0.16|1100⟩ - 0.01|1101⟩ - 0.28|1111⟩ 223 0.87109375
5.473243451175968 -0.68301270189222 2.3226757410898826 5.464268394679676 0.1
[False, True, False, False]
0.01|0000⟩ - 0.17|0001⟩ + 0.6|0010⟩ + 0.03|0011⟩ - 0.28|0100⟩ - 0.04|0101⟩ + 0.05|0110⟩ + 0.16|0111⟩ + 0.01|1000⟩ + 0.01|1001⟩ - 0.06|1010⟩ - 0.32|1011⟩ + 0.03|1100⟩ + 0.45|1101⟩ - 0.43|1110⟩ - 0.02|1111⟩ 223 0.87109375
5.473243451175968 -0.6830127018922199 2.3226757410898826 5.464268394679676 0.1
[False, True, False, False]
0.02|0001⟩ + 0.07|0010⟩ - 0.32|0011⟩ - 0.02|0100⟩ + 0.44|0101⟩ - 0.44|0110⟩ + 0.03|0111⟩ - 0.02|1000⟩ - 0.16|1001⟩ + 0.6|1010⟩ - 0.03|1011⟩ - 0.28|1100⟩ + 0.02|1101⟩ - 0.07|1110⟩ + 0.16|1111⟩ 223 0.87109

## Resource estimates for 1D Ising model using generic SELECT / PREPARE 

In [ ]:
from qualtran.cirq_interop.t_complexity_protocol import t_complexity

num_sites: int = 200
eps: float = 1e-5
m_bits: int = 14

walk = get_walk_operator_for_1d_Ising_model(num_sites, eps)

circuit = cirq.Circuit(phase_estimation(walk, m=m_bits))
%time result = t_complexity(circuit[1:-1])
print(result)

## Resource estimates for 2D Hubbard model using specialized SELECT / PREPARE 
Phase estimation of walk operator for 2D Hubbard Model using SELECT and PREPARE circuits from Section V of https://arxiv.org/abs/1805.03662

In [ ]:
x_dim, y_dim = 20, 20
t = 20
mu = 4 * t
N = x_dim * y_dim * 2
qlambda = 2 * N * t + (N * mu) // 2
delta_E = t / 100
m_bits = int(np.log2(qlambda * np.pi * np.sqrt(2) / delta_E))
walk = get_walk_operator_for_hubbard_model(x_dim, y_dim, t, mu)
circuit = cirq.Circuit(phase_estimation(walk, m=m_bits))
%time result = t_complexity(circuit[1:-1])
print(result)